In [1]:
import requests as r
import pandas as pd
from bs4 import BeautifulSoup

In [2]:

PAP= pd.DataFrame()


for i in range(1,15):
    url=f'https://www.pap.fr/annonce/locations-appartement-paris-75-g439g39073g39154-du-studio-au-2-pieces-{i}'
    headers=f"""Accept: text/html, */*; q=0.01
Accept-Encoding: gzip, deflate, br
Accept-Language: en-US,en;q=0.9
Cache-Control: no-cache
Connection: keep-alive
Cookie: lang=fr; acceptcookies=1; cnil=1; derniere_recherche=%7B%22produit%22%3A%22location%22%2C%22typesbien%22%3A%5B%22appartement%22%5D%2C%22nb_pieces%22%3A%7B%22min%22%3A1%2C%22max%22%3A2%7D%2C%22nb_chambres%22%3A%7B%22min%22%3Anull%2C%22max%22%3Anull%7D%2C%22prix%22%3A%7B%22min%22%3Anull%2C%22max%22%3Anull%7D%2C%22surface%22%3A%7B%22min%22%3Anull%2C%22max%22%3Anull%7D%2C%22surface_terrain%22%3A%7B%22min%22%3Anull%2C%22max%22%3Anull%7D%2C%22geo_objets%22%3A%5B439%2C39073%2C39154%5D%7D
Host: www.pap.fr
Pragma: no-cache
Referer: https://www.pap.fr/annonce/locations-appartement-paris-75-g439g39073g39154-du-studio-au-2-pieces-{i-1}
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-origin
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.66 Safari/537.36
X-Requested-With: XMLHttpRequest"""
    headers=dict([i.split(': ') for i in headers.split('\n')])
    soup=BeautifulSoup(r.get(url, headers=headers).content)
    location=[i.text.strip("\n\t ") for i in soup.select('div.item-body span.h1')]
    rooms=[i.select('li:contains("pièce")')[0].text[:1] if i.select('li:contains("pièce")') else "0" for i in soup.select('ul.item-tags')]
    sqm=[i.select('li:has(small)')[0].text.rstrip('m2') if i.select('li:has(small)') else "0" for i in soup.select('ul.item-tags')]
    bedrooms=[[j.text.rstrip(' chambre').rstrip(' chambres') for j in i.select('li:contains("chambre")')][0] if [j.text for j in i.select('li:contains("chambre")')] else "0" for i in soup.select('ul.item-tags')]
    rental_price = [i.text.strip("\n\t€").replace('\xa0','') for i in soup.select('span.item-price')]
    description=[i.text.strip("\n\t ").replace('\n',' ') for i in soup.select('p.item-description')]
    links=[i.get('href') for i in soup.select('div.item-body a')]
    df = {'Location':location , 'NumberofRooms': rooms , 'Space_sqm': sqm, 'NumberofBedrooms': bedrooms, 'RentalPrice_€': rental_price, 'Description':description, 'Links':links } 
    df = pd.DataFrame.from_dict(df) 
    PAP=PAP.append(df)
    print('page number', i, 'is done')


page number 1 is done
page number 2 is done
page number 3 is done
page number 4 is done
page number 5 is done
page number 6 is done
page number 7 is done
page number 8 is done
page number 9 is done
page number 10 is done
page number 11 is done
page number 12 is done
page number 13 is done
page number 14 is done


In [3]:
PAP.isna().sum()

Location            0
NumberofRooms       0
Space_sqm           0
NumberofBedrooms    0
RentalPrice_€       0
Description         0
Links               0
dtype: int64

In [4]:
PAP['NumberofBedrooms'].str.find('chambres').astype(int)

0    -1
1    -1
2    -1
3    -1
4    -1
     ..
10   -1
11   -1
12   -1
13   -1
14   -1
Name: NumberofBedrooms, Length: 210, dtype: int32

In [5]:
PAP.dtypes

Location            object
NumberofRooms       object
Space_sqm           object
NumberofBedrooms    object
RentalPrice_€       object
Description         object
Links               object
dtype: object

In [6]:
PAP

,Location,NumberofRooms,Space_sqm,NumberofBedrooms,RentalPrice_€,Description,Links
0,Paris 17E (75017),1,16,1,750,Porte Maillot tout près du métro très bien des...,/annonces/appartement-paris-17e-75017-r433800017
1,Paris 2E (75002),2,30,1,1.000,"Appartement 30m2, rue Saint Denis 6ème étag...",/annonces/appartement-paris-2e-75002-r434301833
2,Paris 18E (75018),2,35,1,1.400,Magnifique appartement meublé 2 pièces 35m². A...,/annonces/appartement-paris-18e-75018-r435000161
3,Paris 2E (75002),1,12,0,695,"**Studio calme et lumineux**, situé au 6e étag...",/annonces/appartement-paris-2e-75002-r433201605
4,Paris 5E,2,50,1,2.300,Uniquement Bail de résidence secondaire (pas d...,/annonces/appartement-paris-5e-r431900324
...,...,...,...,...,...,...,...
10,Paris 10E,1,29,0,1.250,"Grand studio meublé de 28,5m2 entièrement équi...",/annonces/appartement-paris-10e-r425800947
11,Paris 13E,2,37,1,1.300,Libre au 15 janvier (éventuellement dès le 13...,/annonces/appartement-paris-13e-r423200172
12,Paris 14E,2,32,1,1.280,"Métro Alésia, proche commerces. 2 pièces 32 m...",/annonces/appartement-paris-14e-r401801556
13,Paris 14E,2,44,1,1.750,"Métro Denfert-Rochereau, Montparnasse, rue Dag...",/annonces/appartement-paris-14e-r434401199


In [7]:
PAP.nunique()

Location             45
NumberofRooms         2
Space_sqm            52
NumberofBedrooms      2
RentalPrice_€       107
Description         202
Links               202
dtype: int64

In [8]:
PAP['NumberofBedrooms'].str.find('chambres').sum() # failure

-210

In [9]:
import re

In [10]:
PAP.Location.replace(r'\([0-9]{5}\)','',regex=True, inplace = True) # adding '^'in the front and '$' at the end only for the exact length of character you are searching in the string.

In [30]:
PAP.Location.value_counts()

Paris 15E                             13
Paris 16E                             12
Paris 16E                             11
Paris 17E                             11
Paris 17E                             10
Paris 13E                             10
Paris 18E                              8
Paris 5E                               7
Paris                                  7
Paris 11E                              7
Paris 8E                               6
Paris 14E                              6
Paris 20E                              6
Paris 8E                               5
Paris 13E                              5
Paris 15E                              5
Paris 3E                               5
Paris 10E                              5
Paris 2E                               5
Paris 10E                              5
Paris 18E                              5
Paris 5E                               5
Paris 7E                               4
Paris 1Er                              4
Paris 19E       

In [28]:
map(str.split, Location)

NameError: name 'Location' is not defined

In [16]:
def clean_currency(x):
    """ If the value is a string, then remove delimiters otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str): #https://www.programiz.com/python-programming/methods/built-in/isinstance
        return(x.replace('.', ''))
    return(x) # if x is not string, return x

In [29]:
PAP['RentalPrice_€'] = PAP['RentalPrice_€'].apply(clean_currency).astype('float')

In [18]:
# ISSUE
split_data = PAP["Location"].str.split(" ")
data = split_data.to_list()
names = ["City", "Arrondissement","Zipcode"]
new_df = pd.DataFrame(data, columns=names)

In [19]:
PAP['RentalPrice_€']

0      750.0
1     1000.0
2     1400.0
3      695.0
4     2300.0
       ...  
10    1250.0
11    1300.0
12    1280.0
13    1750.0
14    1550.0
Name: RentalPrice_€, Length: 210, dtype: float64

In [20]:
# PAP.drop(columns='Location', axis=1)

In [21]:
# PAP['Arrondissement']=PAP['Arrondissement'].str.rstrip('r')

In [22]:
PAP.to_csv('PAP1',sep=";")

In [23]:
import pymysql
from sqlalchemy import create_engine
import getpass

In [24]:
Hostname='localhost'
username='root'
password=getpass.getpass()
database='pap'

········


In [25]:
engine=create_engine(f"mysql+pymysql://{username}:{password}@{Hostname}/{database}")

In [27]:
#ISSUE:
PAP.to_sql('pap6', engine)

In [189]:
# test=[[1,2], [23,3],[17,24]]
# [i[0] for i in test]

In [158]:
# rental_price = [i.text.strip("\n\t").replace('\xa0','') for i in soup.select('span.item-price')]
# [i.select('li:contains("pièce")')[0].text if i.select('li:contains("pièce")') else "unknown" for i in soup.select('ul.item-tags')]

In [162]:
# description=[i.text.strip("\n\t ").replace('\n',' ') for i in soup.select('p.item-description')]
# [[j.text for j in i.select('li:contains("chambre")')][0] if [j.text for j in i.select('li:contains("chambre")')] else "0" for i in soup.select('ul.item-tags') ]

In [301]:
# links=[i.get('href') for i in soup.select('div.item-body a')]
# [i.select('li:has(small)')[0].text if i.select('li:has(small)') else "unknown" for i in soup.select('ul.item-tags')]

In [ ]:
# [[j.text for j in i.select('li:contains("chambre")')][0] for i in soup.select('ul.item-tags') if [j.text for j in i.select('li:contains("chambre")')]]